In [25]:
import json

import polars as pl

from spells import summon
from spells.enums import ColName, ColType
from spells.columns import ColumnSpec

## Fun with card attributes

This is a sample notebook demonstrating how to define custom card attributes and define functions of card attributes at the row level, which now has first-class support in Spells.

First, a little example of using the json dump to define an arbitrary extension of the raw data. Here we will be unoriginal and grab the mana value, which is already available under `MANA_VALUE`.

In [26]:
ext = [
    ColumnSpec(
        name="mana_value_2",
        col_type=ColType.CARD_ATTR,
        expr=pl.col(ColName.CARD_JSON).map_elements(
            lambda x: json.loads(x)["manaValue"], return_dtype=pl.Float64
        ),
        version="1",  # used for cache signature because we have a non-serializable python function
    ),
]

In [27]:
df = summon("DSK", columns=["mana_value", "mana_value_2", "deck"], extensions = ext)
df.head(10)

name,mana_value,mana_value_2,deck
str,f64,f64,i64
"""Abandoned Campground""",0.0,0.0,114632
"""Abhorrent Oculus""",3.0,3.0,26046
"""Acrobatic Cheerleader""",2.0,2.0,188674
"""Altanak, the Thrice-Called""",7.0,7.0,87285
"""Anthropede""",4.0,4.0,50634
"""Appendage Amalgam""",3.0,3.0,99733
"""Arabella, Abandoned Doll""",2.0,2.0,109499
"""Attack-in-the-Box""",3.0,3.0,21374
"""Balemurk Leech""",2.0,2.0,86847


Now, suppose we want to find out how often drafters take the card with lowest mana value. We will need the mana value of the seen cards in `PACK_CARD` as well as the picked card `PICK`. We use function expression definitions with the `name` argument to achieve this in both cases

In [28]:
ext.extend(
    [
        ColumnSpec(
            name="seen_mana_value",
            col_type=ColType.NAME_SUM,
            expr=lambda name, card_context: pl.when(pl.col(f"pack_card_{name}") > 0)
            .then(card_context[name]['mana_value'])
            .otherwise(1000.0),  # we're going to take the min
        ),
        ColumnSpec(
            name="picked_mana_value",
            col_type=ColType.PICK_SUM,
            expr=lambda name, card_context: card_context[name]['mana_value']
        ),
        ColumnSpec(
            name="picked_least_mana_value",
            col_type=ColType.PICK_SUM,
            expr=lambda names: pl.col('picked_mana_value')
            <= pl.min_horizontal(
                [pl.col(f"seen_mana_value_{name}") for name in names]
            ),
        ),
        ColumnSpec(
            name="picked_least_mana_value_rate",
            col_type=ColType.AGG,
            expr=pl.col("picked_least_mana_value") / pl.col("num_taken"),
        ),
    ]
)

While we're at it, let's grab the average mana value of picked cards. Simple as dividing two aggregated column values.

In [29]:
ext.extend(
    [
        ColumnSpec(
            name="avg_mv_picked",
            col_type=ColType.AGG,
            expr=pl.col("picked_mana_value") / pl.col("num_taken"),
        ),
    ]
)

In [30]:
summon(
    "DSK",
    columns=[
        "num_taken",
        "avg_mv_picked",
        "picked_least_mana_value_rate",
    ],
    group_by=["player_cohort"],
    extensions=ext,
)

player_cohort,num_taken,avg_mv_picked,picked_least_mana_value_rate
str,i32,f64,f64
"""Bottom""",1079884,3.01064,0.175012
"""Middle""",2515295,2.98892,0.17853
"""Other""",2184415,3.007436,0.174103
"""Top""",1303151,2.969419,0.182119


As expected, the best players take cheap cards (slightly) more often!